In [14]:
# 🌦️ Interactive Weather Prediction — Safe & Fast Single File

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [15]:
# 1️⃣ Load Dataset
# -----------------------------
print("\n📂 Loading dataset...")
data = pd.read_csv("weather_data.csv")
data = data.dropna()
print(f"✅ Dataset loaded with {len(data)} rows and {len(data.columns)} columns")


📂 Loading dataset...
✅ Dataset loaded with 1000000 rows and 6 columns


In [16]:
# Encode Location
le = LabelEncoder()
data['Location'] = le.fit_transform(data['Location'])

In [17]:
# Drop Date_Time if exists
if 'Date_Time' in data.columns:
    data = data.drop(columns=['Date_Time'])

In [18]:
# 2️⃣ Add Classification Target
# -----------------------------
def categorize_temp(temp):
    if temp < 10:
        return "Cold"
    elif temp < 25:
        return "Mild"
    else:
        return "Hot"

data['Weather_Category'] = data['Temperature_C'].apply(categorize_temp)

In [19]:
# 3️⃣ Optional: Sample for faster training
# -----------------------------
if len(data) > 2000:
    print("⚡ Large dataset detected — using a sample of 2000 rows for fast training")
    data_sample = data.sample(2000, random_state=42)
else:
    data_sample = data.copy()

⚡ Large dataset detected — using a sample of 2000 rows for fast training


In [20]:
# 4️⃣ Train Models (Once)
# -----------------------------
print("\n⏳ Training models... please wait")

# Regression
X_reg = data_sample.drop(columns=['Temperature_C', 'Weather_Category'])
y_reg = data_sample['Temperature_C']
reg_model = RandomForestRegressor(n_estimators=50, random_state=42)
reg_model.fit(X_reg, y_reg)
print("✅ Regression model trained!")


⏳ Training models... please wait
✅ Regression model trained!


In [21]:

# Classification
X_clf = X_reg.copy()
y_clf = data_sample['Weather_Category']
clf_model = RandomForestClassifier(n_estimators=50, random_state=42)
clf_model.fit(X_clf, y_clf)
print("✅ Classification model trained!")

✅ Classification model trained!


In [22]:
import os
import joblib

try:
    os.makedirs("models", exist_ok=True)
    joblib.dump(reg_model, "models/regression_model.pkl")
    joblib.dump(clf_model, "models/classification_model.pkl")
    joblib.dump(le, "models/labelencoder.pkl")
    print("✅ Models saved successfully in 'models' folder!")
except Exception as e:
    print("❌ Error:", e)

✅ Models saved successfully in 'models' folder!


In [24]:
while True:
    print("\n--- New Prediction ---")
    user_location = input("Enter Location (or 'quit' to exit): ").strip()
    if user_location.lower() == "quit":
        print("👋 Exiting program. Goodbye!")
        break

    try:
        user_humidity = float(input("Enter Humidity (%): "))
        user_precip = float(input("Enter Precipitation (mm): "))
        user_wind = float(input("Enter Wind Speed (km/h): "))
    except ValueError:
        print("❌ Invalid numeric input. Please try again.")
        continue

    # Encode location safely
    loc_value = le.transform([user_location])[0] if user_location in le.classes_ else 0

    input_data = pd.DataFrame([{
        "Location": loc_value,
        "Humidity_pct": user_humidity,
        "Precipitation_mm": user_precip,
        "Wind_Speed_kmh": user_wind
    }])

     # -----------------------------
    # 6️⃣ Predict Both Regression & Classification
    # -----------------------------
    temp_prediction = reg_model.predict(inputmum_data)[0]
    weather_prediction = clf_model.predict(input_data)[0]

    print(f"\n🌡️ Predicted Temperature: {round(temp_prediction, 2)} °C")
    print(f"☁️ Predicted Weather Category: {weather_prediction}")


--- New Prediction ---


Enter Location (or 'quit' to exit):  quit


👋 Exiting program. Goodbye!


In [25]:
import os

print(os.listdir("models"))


['classification_model.pkl', 'labelencoder.pkl', 'regression_model.pkl']


In [26]:
import os
print(os.getcwd())


C:\Users\lenovo pc


In [27]:
import joblib, os

# Make sure these variables already exist:
# reg_model = your regression model
# clf_model = your classification model
# le = your LabelEncoder (used for categories)

# Create the models folder if it doesn't exist
os.makedirs("models", exist_ok=True)

# Save all models freshly
joblib.dump(reg_model, "models/regression_model.pkl")
joblib.dump(clf_model, "models/classification_model.pkl")
joblib.dump(le, "models/labelencoder.pkl")

print("✅ Models saved successfully!")


✅ Models saved successfully!
